In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
!pip install --upgrade huggingface_hub
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [2]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/232.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━ 225.3/232.6 kB 8.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [3]:
!pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.2 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=dcb9aed9b61b829cb5142e40d361e96cbdc88e8c8acf6449b28ebc3fdedb495a
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


## Rule based approach to critique the paper

In [ ]:
import requests
from urllib.parse import quote
import feedparser
import spacy
import re
import os
import PyPDF2

nlp = spacy.load("en_core_web_sm")

OPPOSING_TERMS = {
    "increase": ["decrease", "reduce", "diminish", "lower"],
    "decrease": ["increase", "raise", "enhance", "boost"],
    "improve": ["worsen", "deteriorate", "decline", "degrade"],
    "worsen": ["improve", "enhance", "better", "strengthen"],
    "effective": ["ineffective", "useless", "poor", "weak"],
    "ineffective": ["effective", "successful", "strong", "efficient"],
    "accurate": ["inaccurate", "wrong", "incorrect", "erroneous"],
    "fast": ["slow", "delayed", "sluggish", "gradual"],
    "simple": ["complex", "complicated", "difficult", "intricate"],
    "success": ["failure", "unsuccessful", "defeat", "loss"],
    "reliable": ["unreliable", "unstable", "inconsistent", "faulty"],
    "stable": ["unstable", "volatile", "shaky", "unsteady"],
    "positive": ["negative", "adverse", "unfavorable", "detrimental"],
    "high": ["low", "small", "minimal", "reduced"],
    "large": ["small", "tiny", "minor", "negligible"],
    "useful": ["useless", "ineffective", "falls short"],
    "elegant": ["falls short", "ineffective", "poor"],
}

def simulate_llm_analysis(paragraph, prev_paragraphs):
    issues = []
    paragraph_lower = paragraph.lower()
    words = paragraph.split()
    word_count = len(words)

    print(f"DEBUG: Analyzing '{paragraph}' ({word_count} words)")

    if word_count < 20 and "because" not in paragraph_lower and "due to" not in paragraph_lower:
        issues.append("This statement lacks explanation or supporting details.")

    for i, prev_para in enumerate(prev_paragraphs, 1):
        prev_para_lower = prev_para.lower()
        for term, opposites in OPPOSING_TERMS.items():
            if term in paragraph_lower:
                for opp in opposites:
                    if opp in prev_para_lower:
                        issues.append(f"This statement contradicts an earlier assertion in Paragraph {i} ( '{term}' vs. '{opp}' ).")
            elif term in prev_para_lower:
                for opp in opposites:
                    if opp in paragraph_lower:
                        issues.append(f"This statement contradicts an earlier assertion in Paragraph {i} ( '{term}' vs. '{opp}' ).")

    if issues:
        print(f"DEBUG: Issues found - {issues}")
    else:
        print("DEBUG: No issues detected.")

    return issues if issues else ["No major issues detected."]

def classify_rhetorical_role(paragraph):
    print(f"DEBUG: Classified '{paragraph}' as Statement")
    return "Statement"

def clean_text(text):
    text = re.sub(r'\n\s*\n+', '\n\n', text)
    text = re.sub(r'\t+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'Page \d+ of \d+', '', text)
    text = re.sub(r'CHAPTER \d+\.\s*[A-Z\s]+', '', text)
    text = re.sub(r'\d+\.\d+\.\d+\s*[A-Z][a-z\s]+', '', text)
    ref_idx = text.lower().find("bibliography")
    if ref_idx != -1:
        text = text[:ref_idx]
    return text.strip()

def segment_text(text):
    cleaned_text = clean_text(text)
    paragraphs = [p.strip() for p in cleaned_text.split('\n\n') if p.strip() and len(p.split()) > 3]
    if len(paragraphs) <= 1:
        paragraphs = [p.strip() for p in cleaned_text.split('\n') if p.strip() and len(p.split()) > 3]
        if len(paragraphs) <= 1:
            doc = nlp(cleaned_text)
            paragraphs = [sent.text.strip() for sent in doc.sents if len(sent.text.split()) > 3]

    print("DEBUG: Segmented Paragraphs:")
    for i, p in enumerate(paragraphs, 1):
        print(f"  Paragraph {i}: '{p}'")
    return paragraphs

def critique_document(text):
    paragraphs = segment_text(text)
    feedback = []
    prev_paragraphs = []

    for i, para in enumerate(paragraphs, 1):
        role = classify_rhetorical_role(para)
        if role == "Statement":
            issues = simulate_llm_analysis(para, prev_paragraphs)
            if issues != ["No major issues detected."]:
                feedback.append(f"Paragraph {i} ({role}): \"{para}\" - " + " ".join(issues))
        prev_paragraphs.append(para)

    return feedback

def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() or ""
            return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return ""

def fetch_arxiv_papers(query, max_results=1):
    """Fetch papers from arXiv API."""
    base_url = "http://export.arxiv.org/api/query?"
    search_query = f"search_query={quote(query)}&start=0&max_results={max_results}"
    response = requests.get(base_url + search_query)

    if response.status_code != 200:
        raise Exception(f"Failed to fetch from arXiv: {response.status_code}")

    feed = feedparser.parse(response.content)
    papers = []
    for entry in feed.entries:
        pdf_url = entry.link.replace("abs", "pdf") + ".pdf"
        papers.append({
            "title": entry.title,
            "pdf_url": pdf_url,
            "id": entry.id.split("/")[-1]
        })
    return papers

def download_pdf(pdf_url, filename):
    """Download PDF from URL."""
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        return filename
    raise Exception(f"Failed to download PDF: {response.status_code}")

def fetch_and_critique_papers(query, max_results=10):
    try:
        papers = fetch_arxiv_papers(query, max_results)
        if not papers:
            print("No papers found for the query.")
            return

        output_dir = "papers"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        for i, paper in enumerate(papers, 1):
            print(f"\nProcessing Paper {i}: {paper['title']}")
            pdf_path = os.path.join(output_dir, f"{paper['id']}.pdf")

            try:
                downloaded_path = download_pdf(paper['pdf_url'], pdf_path)
                print(f"Downloaded: {downloaded_path}")

                text = extract_text_from_pdf(downloaded_path)
                if text:
                    print(f"\nCritique for '{paper['title']}':")
                    feedback = critique_document(text)
                    if feedback:
                        for line in feedback:
                            print(line)
                    else:
                        print("No issues detected in the document.")
                else:
                    print("No text extracted from the PDF.")
            except Exception as e:
                print(f"Skipping critique due to download failure: {e}")
    except Exception as e:
        print(f"Error fetching papers: {e}")

if __name__ == "__main__":
    query = "machine learning optimization"
    fetch_and_critique_papers(query, max_results=10)

Streaming output truncated to the last 5000 lines.
  Paragraph 249: 'All sub-graphs share weights of layer opera- tions used in cell neural networks.'
  Paragraph 250: 'structured graph, in which nodes are neural architectures, and edges denote morphism operations changing the archi- tecture of the parent node.'
  Paragraph 251: 'A∗search conducts exploita- tion by expanding the architecture of the best node in the tree while simulated annealing performs exploration, which randomly chooses other nodes for expansion with certain predeﬁned probabilities.'
  Paragraph 252: 'Evolutionary algorithm [ /one.oldstyle/three.oldstyle] is also proposed to address the challenge of searching in discrete space.'
  Paragraph 253: 'It mutates Nneural architectures, which have higher acquisition values by random Nlayer operations, evaluates the acquisition of the new ones, and repeats the above steps until the prescribed condition.'
  Paragraph 254: 'Upon solving the two chal- lenges, we could follow t

In [ ]:
import shutil
from google.colab import files


shutil.make_archive('dataset_sample_papers', 'zip', '/content/papers')


files.download('dataset_sample_papers.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## LLM Approcach using the base model without any finetuning of the model but prompt trained and rule based to improve the accuracy


In [ ]:
import requests
import json
import re
import os
import PyPDF2
import feedparser
from urllib.parse import quote

# User credentials for Hugging Face
HF_API_TOKEN = "hf_token"  # Hugging Face token
HF_MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"  # Replace with the actual model name or path

# Hugging Face API URL
HF_API_URL = f"https://api-inference.huggingface.co/models/{HF_MODEL_NAME}"

HEADERS = {
    "Authorization": f"Bearer {HF_API_TOKEN}",
    "Content-Type": "application/json"
}

def call_huggingface_api(prompt):
    payload = {
        "inputs": f"<s>[INST] {prompt} [/INST]</s>",
        "options": {
            "use_cache": True,
            "wait_for_model": True
        }
    }
    try:
        response = requests.post(url=HF_API_URL, headers=HEADERS, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()
        if isinstance(result, list) and "generated_text" in result[0]:
            return result[0]["generated_text"].split("[/INST]")[-1].strip()
        elif isinstance(result, dict) and "error" in result:
            return f"Error from HF: {result['error']}"
        return json.dumps(result)
    except requests.RequestException as e:
        print(f"Error calling Hugging Face API: {e}")
        return "Error: Unable to get response from Hugging Face."

def clean_text(text):
    text = re.sub(r'\n\s*\n+', '\n\n', text)
    text = re.sub(r'\t+', ' ', text)
    ref_idx = text.lower().find("bibliography")
    if ref_idx != -1:
        text = text[:ref_idx]
    return text.strip()

def process_pages(text_with_pages):
    pages = []
    start_processing = False

    for page_num, page_text in text_with_pages:
        cleaned_text = clean_text(page_text)

        # Skip contents page and start from Introduction
        if not start_processing:
            if "contents" in cleaned_text.lower():
                print(f"Skipping Page {page_num}: Detected as Contents page")
                continue
            if "introduction" in cleaned_text.lower():
                start_processing = True
                print(f"Starting from Page {page_num}: Introduction detected")

        if start_processing and cleaned_text:
            section_match = re.search(r'(\d+(\.\d+)*)\s+([A-Za-z\s]+)', cleaned_text)
            section = section_match.group(0) if section_match else "Unknown"
            pages.append((page_num, section, cleaned_text))

    print("DEBUG: Processed Pages:")
    for i, (page, section, text) in enumerate(pages, 1):
        print(f"  Page {page}, Section {section} - '{text[:100]}...' (Word count: {len(text.split())})")
    return pages

def critique_with_llm(page_text, page_num, section, prev_pages, page_idx):
    prompt = f"""Critique the entire text from this page of an academic paper for significant issues only (e.g., major lack of explanation or clear contradictions with prior text), ignoring minor issues like short sentences or missing 'because': '{page_text}' (Previous pages: {' '.join([p[2] for p in prev_pages[:3]])}). Output in this format:
- If an issue exists: "Issue: [Description of significant issue]." followed by "Reason: [Detailed reason]."
- If no issue: "Issue: None detected." followed by "Reason: The page is adequately explained and consistent with prior text."
Focus on major clarity or consistency problems rather than strict detail requirements."""
    response = call_huggingface_api(prompt)
    print(f"DEBUG: LLM Response for Page {page_num}, Section {section}: {response}")
    return response

def critique_document(text_with_pages):
    pages = process_pages(text_with_pages)
    feedback = []
    prev_pages = []

    for i, (page_num, section, page_text) in enumerate(pages, 1):
        critique = critique_with_llm(page_text, page_num, section, prev_pages, i)
        feedback.append((page_num, section, page_text, critique))
        prev_pages.append((page_num, section, page_text))

    return feedback

def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text_with_pages = []
            for page_num, page in enumerate(reader.pages, 1):
                text = page.extract_text() or ""
                text_with_pages.append((page_num, text))
            return text_with_pages
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return []

def fetch_arxiv_papers(query, max_results=1):
    base_url = "http://export.arxiv.org/api/query?"
    search_query = f"search_query={quote(query)}&start=0&max_results={max_results}"
    response = requests.get(base_url + search_query)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch from arXiv: {response.status_code}")
    feed = feedparser.parse(response.content)
    papers = []
    for entry in feed.entries:
        pdf_url = entry.link.replace("abs", "pdf") + ".pdf"
        papers.append({"title": entry.title, "pdf_url": pdf_url, "id": entry.id.split("/")[-1]})
    return papers

def download_pdf(pdf_url, filename):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        return filename
    raise Exception(f"Failed to download PDF: {response.status_code}")

def fetch_and_critique_papers(query, max_results=10):
    try:
        papers = fetch_arxiv_papers(query, max_results)
        if not papers:
            print("No papers found for the query.")
            return
        output_dir = "papers"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        for i, paper in enumerate(papers, 1):
            print(f"\nProcessing Paper {i}: {paper['title']}")
            pdf_path = os.path.join(output_dir, f"{paper['id']}.pdf")
            try:
                downloaded_path = download_pdf(paper['pdf_url'], pdf_path)
                print(f"Downloaded: {downloaded_path}")
                text_with_pages = extract_text_from_pdf(downloaded_path)
                if text_with_pages:
                    print(f"\nCritique for '{paper['title']}':")
                    feedback = critique_document(text_with_pages)
                    for page_num, section, page_text, critique in feedback:
                        print(f"Page {page_num}, Section {section}")
                        print(f"Text: \"{page_text}\"")
                        print(critique)
                        print()
                    if not feedback:
                        print("No issues detected in the document.")
                else:
                    print("No text extracted from the PDF.")
            except Exception as e:
                print(f"Skipping critique due to download failure: {e}")
    except Exception as e:
        print(f"Error fetching papers: {e}")

if __name__ == "__main__":
    query = "machine learning optimization"
    fetch_and_critique_papers(query, max_results=1)



Processing Paper 1: Lecture Notes: Optimization for Machine Learning
Downloaded: papers/1909.03550v1.pdf

Critique for 'Lecture Notes: Optimization for Machine Learning':
Skipping Page 5: Detected as Contents page
Skipping Page 6: Detected as Contents page
Skipping Page 7: Detected as Contents page
Skipping Page 8: Detected as Contents page
Skipping Page 10: Detected as Contents page
Starting from Page 11: Introduction detected
DEBUG: Processed Pages:
  Page 11, Section 1
Introduction
The topic of this lecture series is the mathematical optimization approach
to machine learning - 'Chapter 1
Introduction
The topic of this lecture series is the mathematical optimization approach
to...' (Word count: 291)
  Page 12, Section 4 CHAPTER  - '4 CHAPTER 1. INTRODUCTION
functionality. Indeed, one of the most popular machines are arti
cial neur...' (Word count: 342)
  Page 13, Section 5
goal is to 
nd a hyperplane separating the two classes of vectors - '1.1. EXAMPLES OF OPTIMIZATION PROBLEMS I

In [ ]:

from huggingface_hub import notebook_login

# Login to Hugging Face
notebook_login()

In [ ]:
!pip install torch transformers datasets accelerate bitsandbytes peft


## Fine tuning the base model with a very rough dataset created using ChatGPT

In [5]:
!pip install transformers accelerate datasets peft bitsandbytes trl
!pip install git+https://github.com/huggingface/transformers.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
  Clon

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/dataset_json.jsonl")


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = dataset['train'].train_test_split(test_size=0.1)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0},  # Automatically splits across available devices (GPU+CPU)
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

# QLoRA setup
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 13,631,488 || all params: 7,255,363,584 || trainable%: 0.1879


In [ ]:
def format_function(example):
    return {
        "text": f"{example['input']}\nOutput: {example['output']}"
    }

tokenized_dataset = dataset.map(format_function).map(
    lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=512),
    batched=True
)


Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [ ]:
!pip install -U trl



In [ ]:
from transformers import TrainingArguments  # correct import
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./mistral-qlora-critique",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    fp16=True,  # Use fp16 instead of bf16 in Colab (especially on T4 GPUs)
)


trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
)


trainer.train()


Truncating train dataset:   0%|          | 0/72 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.865200
20,1.120600


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=27, training_loss=1.6441483144406919, metrics={'train_runtime': 419.238, 'train_samples_per_second': 0.515, 'train_steps_per_second': 0.064, 'total_flos': 4727337929146368.0, 'train_loss': 1.6441483144406919})

In [ ]:
trainer.evaluate()


{'eval_loss': 0.5955296158790588,
 'eval_runtime': 3.56,
 'eval_samples_per_second': 2.247,
 'eval_steps_per_second': 0.281}

In [ ]:
repo_id = "Manoghn/mistral-qlora-critique"
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)


README.md:   0%|          | 0.00/71.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.5M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Manoghn/mistral-qlora-critique/commit/13bc21573d815035140c405871fae9cdad482b5a', commit_message='Upload tokenizer', commit_description='', oid='13bc21573d815035140c405871fae9cdad482b5a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Manoghn/mistral-qlora-critique', endpoint='https://huggingface.co', repo_type='model', repo_id='Manoghn/mistral-qlora-critique'), pr_revision=None, pr_num=None)

In [ ]:
trainer.save_model("./mistral-qlora-critique")
tokenizer.save_pretrained("./mistral-qlora-critique")


('./mistral-qlora-critique/tokenizer_config.json',
 './mistral-qlora-critique/special_tokens_map.json',
 './mistral-qlora-critique/tokenizer.model',
 './mistral-qlora-critique/added_tokens.json',
 './mistral-qlora-critique/tokenizer.json')

## Fine tuning the base model using the "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTTrainer
import torch

# Load dataset (your format is different from the original code)
train_data = load_dataset("json", data_files="/content/train_mistral_format.json")["train"]
val_data = load_dataset("json", data_files="/content/val_mistral_format.json")["train"]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:

from huggingface_hub import notebook_login

# Login to Hugging Face
notebook_login()

In [ ]:
# Use Mistral Instruct for better instruction following
model_name = "mistralai/Mistral-7B-Instruct-v0.1"


In [ ]:
# Setup tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

# Prepare model for training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Setup LoRA
lora_config = LoraConfig(
    r=16,  # Reduced from 32/64 to save memory
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Target fewer modules to save memory
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./mistral-qlora-critique",
    per_device_train_batch_size=1,  # Reduced to 1 for Colab memory constraints
    gradient_accumulation_steps=8,  # Increased to maintain effective batch size
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
)

In [ ]:
# For older versions of TRL, we need to manually format the dataset
def format_dataset(dataset):
    """Format a dataset with 'messages' field to a simple text format"""
    formatted_texts = []

    for example in dataset:
        # Extract messages
        messages = example['messages']

        # Get user and assistant content
        user_content = next(msg['content'] for msg in messages if msg['role'] == 'user')
        assistant_content = next(msg['content'] for msg in messages if msg['role'] == 'assistant')

        # Format as a simple text string (Mistral chat format)
        formatted_text = f"<s>[INST] {user_content} [/INST] {assistant_content}</s>"
        formatted_texts.append(formatted_text)

    # Create a new dataset with the formatted texts
    return dataset.add_column("text", formatted_texts)

# Apply formatting to datasets
train_data_formatted = format_dataset(train_data)
val_data_formatted = format_dataset(val_data)

# Now use a regular Trainer with a DataCollator
from transformers import Trainer, DataCollatorForLanguageModeling

# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=1024)

train_data_tokenized = train_data_formatted.map(tokenize_function, batched=True)
val_data_tokenized = val_data_formatted.map(tokenize_function, batched=True)

# Set up proper columns for training
train_data_tokenized.set_format("torch", columns=["input_ids", "attention_mask"])
val_data_tokenized.set_format("torch", columns=["input_ids", "attention_mask"])

# Use regular Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_tokenized,
    eval_dataset=val_data_tokenized,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Train the model
trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.703700
20,1.432600
30,1.366600
40,1.318400
50,1.280600
60,1.220400
70,1.199900
80,1.211800
90,1.183500
100,1.158900


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=168, training_loss=1.2415999514716012, metrics={'train_runtime': 5315.2617, 'train_samples_per_second': 0.254, 'train_steps_per_second': 0.032, 'total_flos': 5.824805571919872e+16, 'train_loss': 1.2415999514716012, 'epoch': 2.96})

In [ ]:
# Save the model
model.save_pretrained("./mistral-critique-model")
tokenizer.save_pretrained("./mistral-critique-model")

('./mistral-critique-model/tokenizer_config.json',
 './mistral-critique-model/special_tokens_map.json',
 './mistral-critique-model/chat_template.jinja',
 './mistral-critique-model/tokenizer.model',
 './mistral-critique-model/added_tokens.json',
 './mistral-critique-model/tokenizer.json')

## Testing the final model trained on dataset generated using the "meta-llama/Meta-Llama-3-8B-Instruct" (Final implementation)

In [ ]:
import PyPDF2
import re
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import PeftModel
from google.colab import files

# Repository ID on Hugging Face
HF_REPO_ID = "Manoghn/mistral-qlora-critique"  # Replace with your actual Hugging Face username and repo name
BASE_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.1"  # Base model for PEFT
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def load_fine_tuned_model_and_tokenizer(hf_repo_id, base_model_id):
    """Load the fine-tuned Mistral model and tokenizer from Hugging Face with 4-bit quantization for GPU."""
    try:
        # Load tokenizer from Hugging Face
        tokenizer = AutoTokenizer.from_pretrained(hf_repo_id)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        if torch.cuda.is_available():
            print("Loading model on GPU with 4-bit quantization...")
            # Enable 4-bit quantization to reduce memory usage
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True
            )

            # Load base model with quantization
            base_model = AutoModelForCausalLM.from_pretrained(
                base_model_id,
                quantization_config=quantization_config,
                device_map="auto"
            )

            # Load PEFT adapters from Hugging Face
            model = PeftModel.from_pretrained(base_model, hf_repo_id)
        else:
            print("CUDA not available, loading model on CPU without quantization.")
            base_model = AutoModelForCausalLM.from_pretrained(
                base_model_id,
                torch_dtype=torch.float32,
                device_map="cpu"
            )
            model = PeftModel.from_pretrained(base_model, hf_repo_id)

        model.eval()
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model or tokenizer: {e}")
        raise

def trim_to_complete_sentence(text):
    """Trim the text to the last complete sentence to avoid abrupt endings."""
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    if sentences:
        return ' '.join(sentences[:-1]) if not text.endswith(('.', '!', '?')) else text
    return text

def critique_paragraph(paragraph, model, tokenizer, max_new_tokens=512):
    """Generate critiques for a given paragraph using the fine-tuned model."""
    try:
        # Modified prompt to make the model less strict
        prompt = f"""Review and critique the following research paper paragraph. Focus on significant issues such as major logical flaws, critical missing evidence, or clear contradictions, while ignoring minor issues like lack of citations or small gaps in detail unless they significantly impact the argument. Provide a balanced critique that acknowledges strengths where appropriate:\n\n{paragraph}"""

        # Format as chat for Mistral
        messages = [{"role": "user", "content": prompt}]

        # Apply chat template with increased max_length for input
        inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", max_length=1024, truncation=True).to(DEVICE)

        # Generate response with adjusted parameters
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.9,
                top_p=0.95,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

        # Decode response
        response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
        # Trim to the last complete sentence
        response = trim_to_complete_sentence(response)
        return response
    except Exception as e:
        print(f"Error generating critique: {e}")
        return "Error: Unable to generate critique."

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file and split into paragraphs."""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num, page in enumerate(reader.pages, 1):
                page_text = page.extract_text() or ""
                text += f"\n\nPage {page_num}:\n{page_text}"

        # Clean text and split into paragraphs
        text = re.sub(r'\n\s*\n+', '\n\n', text)
        paragraphs = [para.strip() for para in text.split('\n\n') if para.strip()]
        return paragraphs
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return []

def critique_pdf(pdf_path, model, tokenizer):
    """Critique each paragraph in the PDF."""
    paragraphs = extract_text_from_pdf(pdf_path)
    if not paragraphs:
        print("No text extracted from the PDF.")
        return

    print(f"\nFound {len(paragraphs)} paragraphs in the PDF. Critiquing each paragraph...\n")
    critiques = []
    for i, paragraph in enumerate(paragraphs, 1):
        print(f"Critiquing Paragraph {i}/{len(paragraphs)}...")
        critique = critique_paragraph(paragraph, model, tokenizer, max_new_tokens=256)
        critiques.append((paragraph, critique))

    # Display results
    for i, (paragraph, critique) in enumerate(critiques, 1):
        print(f"\nParagraph {i}:")
        print(paragraph[:500] + "..." if len(paragraph) > 500 else paragraph)
        print(f"\nCritique {i}:")
        print(critique)
        print("-" * 80)

# Main execution
if __name__ == "__main__":
    # Load model and tokenizer from Hugging Face
    model, tokenizer = load_fine_tuned_model_and_tokenizer(HF_REPO_ID, BASE_MODEL_ID)
    print(f"Model loaded successfully on {DEVICE}")

    # Upload PDF
    print("Please upload the PDF file:")
    uploaded = files.upload()
    if uploaded:
        pdf_path = list(uploaded.keys())[0]
        print(f"Uploaded PDF: {pdf_path}")
        critique_pdf(pdf_path, model, tokenizer)
    else:
        print("No file uploaded.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Model loaded successfully on cuda
Please upload the PDF file:


Saving Handwritten Equation Solver 1.pdf to Handwritten Equation Solver 1.pdf
Uploaded PDF: Handwritten Equation Solver 1.pdf


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Found 19 paragraphs in the PDF. Critiquing each paragraph...

Critiquing Paragraph 1/19...
Critiquing Paragraph 2/19...
Critiquing Paragraph 3/19...
Critiquing Paragraph 4/19...
Critiquing Paragraph 5/19...
Critiquing Paragraph 6/19...
Critiquing Paragraph 7/19...
Critiquing Paragraph 8/19...
Critiquing Paragraph 9/19...
Critiquing Paragraph 10/19...
Critiquing Paragraph 11/19...
Critiquing Paragraph 12/19...
Critiquing Paragraph 13/19...
Critiquing Paragraph 14/19...
Critiquing Paragraph 15/19...
Critiquing Paragraph 16/19...
Critiquing Paragraph 17/19...
Critiquing Paragraph 18/19...
Critiquing Paragraph 19/19...

Paragraph 1:
Page 1:
              Handwritten Equation Solver

Critique 1:
The paragraph provides a clear and concise overview of the topic, stating that the research is focused on developing a handwritten equation solver. The introduction is well-structured, with the use of headings and bullet points to break up the text and make it more readable. The language is straigh